In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential, Model, load_model

import tensorflow as tf

import pandas as pd

import os
import pickle
import numpy as np

import scipy.sparse as sp
import scipy.io as spio

import isolearn.io as isoio
import isolearn.keras as iso


Using TensorFlow backend.


In [2]:
#Load sequence data

df = pd.read_csv('../../../aparent/data/prepared_data/apa_leslie_derti_apadb_pair_data/apa_leslie_derti_apadb_pair_data_df_pair.csv', sep='\t')


In [3]:
#Load APARENT Resnet

model_name = 'aparent_all_libs_resnet_no_clinvar_wt_ep_5'

save_dir = os.path.join(os.getcwd(), '../../../aparent-resnet/saved_models')
model_path = os.path.join(save_dir, model_name + '.h5')

aparent_model = load_model(model_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/jlinder2/anaconda3/envs/tensorflow/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
#Score all sequences with APARENT (use sum of cuts to capture OR-like logic)

max_n_pas = 10

encoder = iso.OneHotEncoder(205)

x_prox = np.concatenate([encoder.encode(row['wide_seq_ext_prox'][175-70:175-70+205])[None, None, :, :] for _, row in df.iterrows()], axis=0)
x_dist = np.concatenate([encoder.encode(row['wide_seq_ext_dist'][175-70:175-70+205])[None, None, :, :] for _, row in df.iterrows()], axis=0)

fake_lib = np.zeros((x_prox.shape[0], 13))
fake_lib[:, 11] = 1.

#Pad
n_pad = 32 - x_prox.shape[0] % 32 if x_prox.shape[0] % 32 != 0 else 0

fake_lib = np.concatenate([fake_lib, np.zeros((n_pad, 13))], axis=0)
x_prox = np.concatenate([x_prox, np.zeros((n_pad, 1, 205, 4))], axis=0)
x_dist = np.concatenate([x_dist, np.zeros((n_pad, 1, 205, 4))], axis=0)

_, pred_cuts_prox = aparent_model.predict(x=[x_prox, fake_lib], batch_size=32, verbose=1)
_, pred_cuts_dist = aparent_model.predict(x=[x_dist, fake_lib], batch_size=32, verbose=1)

if n_pad > 0 :
    pred_cuts_prox = pred_cuts_prox[:-n_pad, :]
    pred_cuts_dist = pred_cuts_dist[:-n_pad, :]


24704/24704 [==============================] - 235s 10ms/step


In [8]:

isoform_start = 77
isoform_end = 127

pred_iso_prox = np.sum(pred_cuts_prox[:, isoform_start:isoform_end], axis=1)
score_prox = np.log(pred_iso_prox / (1. - pred_iso_prox))

pred_iso_dist = np.sum(pred_cuts_dist[:, isoform_start:isoform_end], axis=1)
score_dist = np.log(pred_iso_dist / (1. - pred_iso_dist))

isoform_start = 0
isoform_end = 205

pred_iso_prox_all_cuts = np.sum(pred_cuts_prox[:, isoform_start:isoform_end], axis=1)
score_prox_all_cuts = np.log(pred_iso_prox_all_cuts / (1. - pred_iso_prox_all_cuts))

pred_iso_dist_all_cuts = np.sum(pred_cuts_dist[:, isoform_start:isoform_end], axis=1)
score_dist_all_cuts = np.log(pred_iso_dist_all_cuts / (1. - pred_iso_dist_all_cuts))

score_prox_apadb_region = []
score_dist_apadb_region = []

i = 0
for _, row in df.iterrows() :
    
    strand = row['strand']
    
    cut_start_prox = 0
    cut_end_prox = 205
    cut_start_dist = 0
    cut_end_dist = 205
    
    if strand == '+' :
        cut_start_prox = row['cut_start_prox'] - row['pas_pos_prox'] + 70
        cut_end_prox = row['cut_end_prox'] - row['pas_pos_prox'] + 70
        cut_start_dist = row['cut_start_dist'] - row['pas_pos_dist'] + 70
        cut_end_dist = row['cut_end_dist'] - row['pas_pos_dist'] + 70
    else :
        cut_start_prox = row['pas_pos_prox'] - row['cut_end_prox'] + 76
        cut_end_prox = row['pas_pos_prox'] - row['cut_start_prox'] + 76
        cut_start_dist = row['pas_pos_dist'] - row['cut_end_dist'] + 76
        cut_end_dist = row['pas_pos_dist'] - row['cut_start_dist'] + 76
    
    pred_iso_p = np.sum(pred_cuts_prox[i, cut_start_prox:cut_end_prox])
    score_p = np.log(pred_iso_p / (1. - pred_iso_p))

    pred_iso_d = np.sum(pred_cuts_dist[i, cut_start_dist:cut_end_dist])
    score_d = np.log(pred_iso_d / (1. - pred_iso_d))
    
    score_prox_apadb_region.append(score_p)
    score_dist_apadb_region.append(score_d)
    
    i += 1

score_prox_apadb_region = np.array(score_prox_apadb_region)
score_dist_apadb_region = np.array(score_dist_apadb_region)


In [9]:
#Copy the dataframe and store isoform predictions

pred_df = df.copy().reset_index(drop=True)

pred_df['score_prox'] = score_prox
pred_df['score_dist'] = score_dist
pred_df['score_prox_all_cuts'] = score_prox_all_cuts
pred_df['score_dist_all_cuts'] = score_dist_all_cuts
pred_df['score_prox_apadb_region'] = score_prox_apadb_region
pred_df['score_dist_apadb_region'] = score_dist_apadb_region


In [10]:
#Dump prediction dataframe and cut probability matrix

isoio.dump({'pred_df' : pred_df}, 'apa_leslie_derti_apadb_pair_data/' + model_name + '_predictions')